
# Everywatch Auction Dashboard — Build & Validation
**Author:** Kamil Tuniewicz  
**Date:** Sep 30, 2025

This notebook documents the data preparation and validation KPIs used for the executive dashboard.


In [ ]:
import pandas as pd, numpy as np, json, pathlib
xlsx = pathlib.Path("../Kamil Tuniewicz_AUCTION DATA.xlsx")  # adjust path if needed
df = pd.read_excel(xlsx, sheet_name=0)
df.columns = [str(c).strip().replace(" ", "_") for c in df.columns]
df.head(3)

## Clean & engineer fields

In [ ]:
need = ["CityName","Price_RangeUSD","MaterialsJson","ColorsJson","ReferenceNumberId","CaseSizeName",
        "MinEstUsd","MaxEstUsd","Category","ManufacturerName","ModelName","RegionName","CountryName",
        "SourceName","SaleEndDate","LotStatusName","RealUSD","Final_SoldPriceUsd","HammerUSD","PerformanceUSD"]
for c in need:
    if c not in df.columns: df[c] = np.nan

df["SaleEndDate"] = pd.to_datetime(df["SaleEndDate"], errors="coerce")
df["year_month"]  = df["SaleEndDate"].dt.to_period("M").astype(str)

for c in ["RealUSD","Final_SoldPriceUsd","HammerUSD","MinEstUsd","MaxEstUsd","PerformanceUSD"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["revenue_usd"] = np.where(df["RealUSD"].notna(), df["RealUSD"],
                        np.where(df["Final_SoldPriceUsd"].notna(), df["Final_SoldPriceUsd"],
                                 np.where(df["HammerUSD"].notna(), df["HammerUSD"], np.nan)))

df["sold_flag"] = (
    df["LotStatusName"].astype(str).str.contains("sold", case=False, na=False)
    | (df["Final_SoldPriceUsd"].fillna(0) > 0)
)

def parse_tokens(val):
    if pd.isna(val): return []
    s = str(val).strip()
    try:
        if s.startswith("[") or s.startswith("{"):
            obj = json.loads(s)
            if isinstance(obj, list):
                return [str(x).strip() for x in obj if str(x).strip()]
            if isinstance(obj, dict):
                out=[]
                for k,v in obj.items():
                    keep = (isinstance(v,bool) and v) or (isinstance(v,(int,float)) and v) or (isinstance(v,str) and v.strip())
                    if keep: out.append(str(k).strip())
                return out
    except Exception:
        pass
    return [t.strip() for t in s.replace("/", ",").split(",") if t.strip()]

df["MaterialsList"] = df["MaterialsJson"].apply(parse_tokens)
df["ColorsList"]    = df["ColorsJson"].apply(parse_tokens)

df.sample(5)[["SourceName","RegionName","CountryName","ManufacturerName","ModelName","CaseSizeName","Price_RangeUSD"]]

## Validation KPIs (should match dashboard Overview)

In [ ]:
lots = len(df)
sold = int(df["sold_flag"].sum())
st   = sold / lots if lots else np.nan
rev  = float(df["revenue_usd"].fillna(0).sum())
avg  = (rev / sold) if sold else np.nan

print(f"Total lots: {lots:,}")
print(f"Sold lots: {sold:,}")
print(f"Sell-through: {st:.1%}" if pd.notna(st) else "Sell-through: —")
print(f"Total revenue (USD): {rev:,.2f}")
print(f"Avg sold price (USD): {avg:,.2f}" if pd.notna(avg) else "Avg sold price: —")

## Ship the signed-off HTML to a `dist/` folder (optional)

In [ ]:
import shutil, pathlib
dist = pathlib.Path("../dist"); dist.mkdir(exist_ok=True)
shutil.copy2("../Everywatch_CEO_Dashboard_RELEASE.html", dist/"Everywatch_CEO_Dashboard_RELEASE.html")
list(dist.iterdir())